# Assignment 1 part 2 - Speech enhancement

### Importing libraries

In [1]:
import librosa
import numpy as np
import tensorflow as tf
from IPython.core.display import HTML

### Setting configurations for GPU

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True

In [3]:
s, sr=librosa.load('train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr=librosa.load('train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)

In [4]:
S_abs = np.abs(S)
X_abs = np.abs(X)

In [5]:
S_abs.shape

(513, 2459)

### Loading the test signals 

In [5]:
s_t_1, sr_1=librosa.load('test_x_01.wav', sr=None)
X_test_1=librosa.stft(s_t_1, n_fft=1024, hop_length=512)
s_t_2, sr_2=librosa.load('test_x_02.wav', sr=None)
X_test_2=librosa.stft(s_t_2, n_fft=1024, hop_length=512)

In [15]:
X = tf.placeholder(tf.float32, shape=[None, 513])
Y = tf.placeholder(tf.float32, shape=[None, 513])
rate = tf.placeholder(tf.float32)
num_nodes_1 = 1000
num_nodes_2 = 1024
num_nodes_3 = 1024
num_nodes_4 = 1024
learning_rate = 0.001
# kernel_initializer=tf.keras.initializers.he_normal(),
#                                           bias_initializer=tf.keras.initializers.he_normal(),

### Defining the network

In [16]:

first_layer = tf.layers.dense(X, num_nodes_1, kernel_initializer=tf.keras.initializers.glorot_normal(),
                               bias_initializer=tf.keras.initializers.glorot_normal(),
                             activation = tf.nn.relu)
# drop_1 = tf.layers.dropout(first_layer,rate = rate)
second_layer = tf.layers.dense(first_layer, num_nodes_1, kernel_initializer=tf.keras.initializers.glorot_normal(),
                               bias_initializer=tf.keras.initializers.glorot_normal(),
                              activation = tf.nn.relu)
# drop_2 = tf.layers.dropout(second_layer,rate = rate)
# third_layer = tf.layers.dense(drop_2, num_nodes_3, kernel_initializer=tf.keras.initializers.glorot_normal(),
#                                           bias_initializer=tf.keras.initializers.glorot_normal(),
#                               activation = tf.nn.tanh)
# drop_3 = tf.layers.dropout(third_layer,rate = rate)
third_layer = tf.layers.dense(second_layer, num_nodes_1,kernel_initializer=tf.keras.initializers.glorot_normal(),
                              bias_initializer = tf.keras.initializers.glorot_normal(),
                              activation = tf.nn.relu)

# drop_4 = tf.layers.dropout(encoded,rate =rate)
# third_layer_1 = tf.layers.dense(drop_4, num_nodes_3,  kernel_initializer=tf.keras.initializers.glorot_normal(),
#                                           bias_initializer=tf.keras.initializers.glorot_normal(),
#                               activation = tf.nn.tanh)

# drop_5 = tf.layers.dropout(third_layer_1,rate =rate)
fourth_layer = tf.layers.dense(third_layer, num_nodes_1,  kernel_initializer=tf.keras.initializers.glorot_normal(),
                                          bias_initializer=tf.keras.initializers.glorot_normal(),
                              activation = tf.nn.relu)
# drop_6 = tf.layers.dropout(second_layer_1,rate =rate)
# first_layer_1 = tf.layers.dense(second_layer_1, 700,  kernel_initializer=tf.contrib.layers.xavier_initializer(),
# #                                           bias_initializer=tf.contrib.layers.xavier_initializer(),
#                               activation = tf.nn.tanh)
# drop_7 = tf.layers.dropout(first_layer_1,rate =rate)
decoded =  tf.layers.dense(fourth_layer, 513,  kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                           bias_initializer=tf.contrib.layers.xavier_initializer(),
                              activation = tf.nn.relu)

# logits = tf.layers.dense(decoded, Y.shape[1], activation=tf.nn.tanh)

# prediction = tf.nn.softmax(logits)

loss_op = tf.losses.mean_squared_error(labels = Y,
                            predictions = decoded)
# loss_op = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
#                                          labels = Y)
# loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
#     logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
# correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# accuracy = tf.sqrt(tf.reduce_mean(loss_op))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    batch_size = 64
    
    for epoch in range(0, 300):
        accuracy_temp = []
        for i in range(0, X_abs.shape[1]+1, batch_size):
            X_batch = X_abs.T[i:i+batch_size]
            Y_batch = S_abs.T[i:i+batch_size]
            accuracy_, _ = sess.run([ loss_op, train_op], feed_dict={X:X_batch, 
                                                          Y:Y_batch,
                                                          rate:0.10})
            accuracy_temp.append(accuracy_)
        accuracy_ = np.array(accuracy_temp).mean()
#         print(str(accuracy_) + 'for epoch '+str(epoch))
        accuracy_temp = []
        if epoch % 100 == 0:
            print('Accuracy is '+str(accuracy_) + 'for epoch '+str(epoch))
    output_test_1 = sess.run(decoded, feed_dict={X:np.abs(X_test_1).T,
                                                 rate:0.0})
    output_test_2 = sess.run(decoded, feed_dict={X:np.abs(X_test_2).T,
                                                 rate:0.0})

Accuracy is 0.061181895for epoch 0
Accuracy is 0.0049223015for epoch 100
Accuracy is 0.0026222828for epoch 200


### Code for displaying the audio file

In [17]:
def wavPlayer(filepath):
    src = """
    <head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">   
    <body>
    <audio controls="controls" style="width:600px" >
      <source src="files/%s" type="audio/wav" />
    </audio>
    </body>
    """%(filepath)
    display(HTML(src))

In [18]:
X_test_phase = X_test_1 / np.abs(X_test_1)
out = np.multiply(X_test_phase, output_test_1.T)
y_hat = librosa.istft(out,  hop_length=512)
y_1 = librosa.istft(X_test_1,  hop_length=512)

In [19]:
librosa.output.write_wav('test_s_01_recons.wav', y_hat, sr_1)

#### Playing the reconstructed test signal 1

In [20]:
wavPlayer("test_s_01_recons.wav")

In [21]:
X_test_phase = X_test_2 / np.abs(X_test_2)
out = np.multiply(X_test_phase, output_test_2.T)
y_hat = librosa.istft(out,  hop_length=512)
y_2 = librosa.istft(X_test_2,  hop_length=512)
librosa.output.write_wav('test_s_02_recons.wav', y_hat, sr_2)

#### Playing the reconstructed test signal 2

In [22]:
wavPlayer("test_s_02_recons.wav")